In [218]:
!pip uninstall -qqy jupyterlab kfp
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

In [219]:
from google import genai
from google.genai import types
from google.api_core import retry
from IPython.display import HTML, Markdown, display
import chromadb
from chromadb.utils.embedding_functions import EmbeddingFunction

In [220]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [233]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret('GOOGLE_API_KEY')
WEATHER_API_KEY = user_secrets.get_secret('WEATHER_API_KEY')
TRAVELAGENT_SYSINT = (
    "You are TravelAIAgent, a friendly and professional travel companion. A human will chat with you while planning or enjoying their trip. "
    "You assist with a few specific services: giving current weather information, sharing local travel tips and cultural insights, offering translation help, "
    "and supporting general travel planning — like suggesting destinations based on preferences. "
    "At the start, the user is shown these available services, so you don't need to repeat them unless asked. "
    "As the user shares information — like their preferences, goals, or plans — you remember it and use it to offer helpful, natural suggestions later. "
    "Respond in a conversational tone, like a helpful travel agent talking to a client. "
    "Do not format your replies like a Markdown document. Avoid bullets or numbered lists unless specifically requested. "
    "Use emojis to make the conversation human-friendly and fun."
    " Don't ask follow-up questions unless the user specifically invites them. Instead, offer helpful suggestions based on what you already know."
)

client = genai.Client(api_key=GOOGLE_API_KEY)

In [234]:
config = types.GenerateContentConfig(
    temperature = 0.7,
    max_output_tokens = 512,
    system_instruction = TRAVELAGENT_SYSINT,
)

# Chatbot functionality

In [235]:
def interpret_user_request(user_input):
    prompt = (
        "You are a function router. Based on the user message, output ONLY a Python dictionary.\n"
        "- If the user is asking for weather info, return: {'intent': 'get_weather', 'location': '<CITY>'}\n"
        "- If the user is asking for tips or local info (scams, etiquette, transport, etc.), return: {'intent': 'get_tip', 'query': '<QUESTION>'}\n"
        "- If the user asks about the weather in last mentioned city, return: {'intent': 'get_weather_last'}"
        "- Otherwise, return: {'intent': 'chat'}\n"
        "Respond with ONLY the dictionary. No explanations, no code blocks.\n\n"
        f"User: {user_input}"
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )

    try:
        return ast.literal_eval(response.text.strip())
    except Exception:
        return {'intent': 'chat'}

def extract_city_from_text(user_input: str) -> str | None:
    prompt = (
        "You are a city name extractor. If the following sentence clearly refers to a known city, "
        "output ONLY the city name as a string, with no extra text.\n"
        "If no city is clearly mentioned, output 'None'.\n\n"
        f"Sentence: {user_input}"
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )

    result = response.text.strip().strip("'\"")
    return None if result.lower() == "none" else result



## Weather info

In [236]:
# Get weather info
import requests

def get_weather(city):
    url = (
        f"http://api.openweathermap.org/data/2.5/weather?"
        f"q={city}&appid={WEATHER_API_KEY}&units=metric"
    )
    response = requests.get(url)
    if response.status_code != 200:
        return f"Sorry, I couldn’t fetch the weather for {city}."
    
    data = response.json()
    description = data["weather"][0]["description"]
    temp = data["main"]["temp"]
    feels_like = data["main"]["feels_like"]

    return (
        f"The current weather in {city} is {description} with a temperature of {temp}°C "
        f"(feels like {feels_like}°C)."
    )

## RAG + ChromaDB

In [237]:
documents = [
    "In Bangkok, visit Chatuchak Market on the weekend and avoid tuk-tuk scams near tourist spots.",
    "In Paris, always greet with 'Bonjour' before asking questions. Most museums are closed on Mondays.",
    "In London, public transport uses contactless cards. Tipping is not expected in pubs.",
    "In Dubai, public displays of affection are discouraged. Dress modestly in public areas.",
    "In Singapore, chewing gum is banned. It’s one of the cleanest and safest cities globally.",
    "In Kuala Lumpur, try nasi lemak for breakfast. Petronas Towers are best viewed at sunset.",
    "In New York City, walk fast, tip 15–20%, and explore boroughs beyond Manhattan.",
    "In Istanbul, try local ferry rides and visit both European and Asian sides of the city.",
    "In Tokyo, avoid loud phone calls on trains. Convenience stores (konbini) have everything.",
    "In Antalya, the old town (Kaleiçi) is walkable and filled with ancient Roman ruins.",
    "In Seoul, Korean street food markets like Gwangjang are must-visit. Public transport is efficient.",
    "In Rome, beware of pickpockets at popular landmarks. Tap water is drinkable.",
    "In Phuket, island tours are best booked locally. Avoid animal-based attractions.",
    "In Mecca, non-Muslims cannot enter the central holy area. Stay hydrated in the heat.",
    "In Hong Kong, use the Octopus card for transport and street food. Don't miss Victoria Peak.",
    "In Barcelona, be cautious of pickpockets on Las Ramblas. Tapas culture is big—dine late.",
    "In Zurich, trains are punctual to the minute. Public water fountains offer safe drinking water.",
    "In Cairo, tipping (baksheesh) is expected for most services. Visit the pyramids early to avoid crowds.",
    "In Sydney, sun protection is essential year-round. Use an Opal card for public transport.",
    "In Marrakech, haggle respectfully in souks. Fridays are holy—many shops may close.",
    "In Amsterdam, watch for bikes at crossings. Many museums require advance reservations.",
    "In Mexico City, avoid tap water. Street tacos are a must, but go where locals eat.",
    "In Athens, the Acropolis is best visited early morning. Tipping is appreciated but not required.",
    "In Vancouver, use contactless or Compass Card for transit. Pack for rain, even in summer.",
    "In Buenos Aires, late dinners and tango shows are local staples. Beware of counterfeit currency.",
    "In Prague, the Old Town is stunning but crowded—explore Žižkov and Letná for a local vibe.",
    "In Vienna, classical concerts abound, but dress semi-formally. Public transport is safe and clean.",
    "In Cape Town, avoid walking after dark in certain areas. Visit Table Mountain on a clear day.",
    "In Lisbon, trams can get crowded—beware of pickpockets. Try pastel de nata from local bakeries.",
    "In Los Angeles, public transport is limited—consider renting a car. Tipping is standard at 20%."
]


In [238]:
# Gemini Embedding Function class
class GeminiEmbeddingFunction(EmbeddingFunction):
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input):
        task_type = "retrieval_document" if self.document_mode else "retrieval_query"
        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(task_type=task_type)
        )
        return [e.values for e in response.embeddings]

# Initialize ChromaDB
chroma_client = chromadb.Client()
DB_NAME = "travel_tips"
embed_fn = GeminiEmbeddingFunction()

db = chroma_client.get_or_create_collection(
    name=DB_NAME,
    embedding_function=embed_fn
)

# Store tips
db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

# Search and answer
def search_and_answer(user_question):
    # Switch to query embedding mode
    embed_fn.document_mode = False

    # Search top matching passage
    result = db.query(query_texts=[user_question], n_results=2)
    passages = result["documents"][0]

    # Create grounded prompt for Gemini
    prompt = (
        "You are a helpful travel assistant. Use the information from the following passages "
        "to answer the user's question clearly. If irrelevant, ignore it.\n\n"
        f"QUESTION: {user_question}\n"
    )

    for p in passages:
        prompt += f"PASSAGE: {p.strip()}\n"

    # Generate response
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Part.from_text(text=prompt)]
    )
    return response.text

# Talk to ChatBot here

In [240]:
onboarding_message = (
    "Hi there! I'm TravelAIAgent — here to help you with anything travel-related.\n\n"
    "Here’s what I can assist with:\n"
    "- Give you real-time weather information for any city\n"
    "- Share travel tips and cultural advice for popular destinations\n"
    "- Translate short pieces of text for better communication\n"
    "- Help you plan your travel or suggest destinations based on your interests\n\n"
    "You can just talk to me naturally. What’s on your mind?"
)
# Initialize chat history and memory
history = []
memory = {"last_city": None}

print("TravelAI:", onboarding_message)
history.append(types.ModelContent(parts=[types.Part.from_text(text=onboarding_message)]))

if not user_memory_db.get()['ids']:
    print("✅ Memory is fully cleared. Starting fresh.")
else:
    print("⚠️ Memory still contains:", user_memory_db.get()['documents'])

while True:
    user_input = input("You: ").strip()
    if user_input.lower() == 'q':
        print("Goodbye!")
        break

    # Step 1: Interpret intent
    action = interpret_user_request(user_input)

    # Step 2: Handle weather by city
    if action.get("intent") == "get_weather" and "location" in action:
        city = action["location"]
        memory["last_city"] = city  # Store the last city
        weather_info = get_weather(city)
        print("AI:", weather_info)
        history.append(types.ModelContent(parts=[types.Part.from_text(text=weather_info)]))
        continue

    # Step 3: Handle travel tips via RAG
    if action.get("intent") == "get_tip" and "query" in action:
        query = action["query"]
        city = extract_city_from_text(query)
        memory["last_city"] = city if city else query
        tip_answer = search_and_answer(query)
        print("AI:", tip_answer)
        history.append(types.ModelContent(parts=[types.Part.from_text(text=tip_answer)]))
        continue


    ## EXTRAS
    if action.get("intent") == "get_weather_last":
        if memory["last_city"]:
            city = memory["last_city"]
            weather_info = get_weather(city)
            print(f"AI: Here's the latest weather in {city}:\n{weather_info}")
            history.append(types.ModelContent(parts=[types.Part.from_text(text=weather_info)]))
        else:
            print("AI: Sorry, I don't know your last mentioned city. Try asking about a city first.")
        continue

    # Step 5: Fallback to regular chat
    history.append(types.UserContent(parts=[types.Part.from_text(text=user_input)]))
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        config=config,
        contents=history
    )
    history.append(response.candidates[0].content)
    print("AI:", response.text)

TravelAI: Hi there! I'm TravelAIAgent — here to help you with anything travel-related.

Here’s what I can assist with:
- Give you real-time weather information for any city
- Share travel tips and cultural advice for popular destinations
- Translate short pieces of text for better communication
- Help you plan your travel or suggest destinations based on your interests

You can just talk to me naturally. What’s on your mind?
✅ Memory is fully cleared. Starting fresh.


You:  What do you know about me?


AI: Right now, I only know that we're chatting and that I can help with weather info, travel tips, translations, and travel planning. ✈️

As we talk, I'll remember your preferences and travel plans so I can give you better suggestions. 😊 Just let me know what you're thinking!



KeyboardInterrupt: Interrupted by user

In [ ]:
Markdown(response.text)